Task 1: Managing Conversation History with Summarization
1. Maintain a running conversation history of user–assistant chats.
2. Implement summarization of conversation history to keep it concise.
3. Allow customization options for truncation:
a. Limit by number of conversation turns (e.g., last n messages).
b. Limit by character/word length.

4. Add periodic summarization:
a. Perform summarization after every k-th run of the conversation.
b. Store/replace the summarized version in the conversation history.

5. Demonstrate the functionality by:
a. Feeding multiple conversation samples.
b. Showing outputs at different truncation settings.
c. Showing how summarization happens after every k-th run (e.g., after every 3rd run).

In [1]:
# %pip install groq

In [2]:
import textwrap
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("XAI_API_KEY")


class ChatManager():
    def __init__(self, truncation,grok_api_key=None):
        self.conversation = []
        self.archive = []
        self.msg_count = 0
        self.truncation = truncation
        self.groq_client=Groq(grok_api_key=api_key) if grok_api_key else None
        self.samples = [
            ("user","Hi, I'm building khush chatbot."),
            ("assistant","Great! What extra features do i need?"),
            ("user","I want truncation and periodic summarization."),
            ("assistant","Do i want summaries to replace my old messages?"),
            ("user", "Yes, and keep an archive."),
            ("assistant", "lets build one."),
            ("user","Yes, i am excited to build one"),
            ("assistant","yes me too lets gooooooo!")
        ]





    def truncate_by_turns(self, conv, n):
        return conv[-n:]

    def truncate_by_chars(self, conv, no_of_chars):
        output = []
        total_char = 0
        for role, text in reversed(conv):
            if total_char + len(text) > no_of_chars:
                remaining = no_of_chars - len(text)
                if remaining > 0:
                    output.append((role, text[-remaining:]))
                break
            output.append((role, text))
            total_char += len(text)
        return list(reversed(output))

    def truncate_by_words(self, conv, no_of_words):
        output = []
        total_word = 0
        for role, text in reversed(conv):
            words = text.split()
            wlen = len(words)
            if wlen + total_word > no_of_words:
                remaining = no_of_words - wlen
                if remaining > 0:
                    output.append((role, " ".join(words[-remaining:])))
                break
            output.append((role, text))
            total_word += wlen
        return list(reversed(output))

    def text_summary(self, msg):
      if self.groq_client:
        sum_lines = [f"{role.upper()}: {text}" for role, text in msg]
        sum_joined = " ".join(sum_lines)
        completion = self.groq_client.chat.completions.create(
                model="llama-3.1-8b-instant",  
                messages=[
                    {"role": "system", "content": "Summarize this chat briefly in 2-3 sentences."},
                    {"role": "user", "content": sum_joined}
                ]
            )
        return completion.choices[0].message.content.strip()
      else:
          sum_lines = [f"{role.upper()}:{text}" for role, text in msg]
          sum_joined = " ".join(sum_lines)
          final_summary = textwrap.shorten(sum_joined, width=150)
          return final_summary

    def add_messages(self, role, text, k=4):
        self.conversation.append((role, text))
        self.msg_count += 1
        if self.msg_count % k == 0:
            trunc_type, value = self.truncation
            if trunc_type == "turns":
                msg = self.truncate_by_turns(self.conversation, value)
            elif trunc_type == "chars":
                msg = self.truncate_by_chars(self.conversation, value)
            elif trunc_type == "words":
                msg = self.truncate_by_words(self.conversation, value)
            else:
                msg = self.conversation
            summary = self.text_summary(msg)
            self.archive.append(msg)  # save truncated part
            self.conversation = [m for m in self.conversation if m not in msg]
            self.conversation.append(("Summary", summary))
            return summary
        return None


truncation_type = input("Enter truncation type (turns,chars,words): ")
truncation_value = int(input("Enter truncation value: "))
truncation = (truncation_type, truncation_value)

cm = ChatManager(truncation)
for role, text in cm.samples:
    s = cm.add_messages(role, text, k=4)
    if s:
        print(f"\nSummarization triggered → {s}\n")

print("\n--- Final Conversation ---")
for role, text in cm.conversation:
    print(f"{role}: {text}")

print("\n--- Archive ---")
for idx, archive_msg in enumerate(cm.archive, 1):
    print(f"Archived #{idx}:")
    for r, t in archive_msg:
        print(f"   {r}: {t}")



Summarization triggered → ASSISTANT:Great! What extra features do i need? USER:I want truncation and periodic summarization. ASSISTANT:Do i want summaries to replace my [...]


Summarization triggered → USER:Yes, and keep an archive. ASSISTANT:lets build one. USER:Yes, i am excited to build one ASSISTANT:yes me too lets gooooooo!


--- Final Conversation ---
user: Hi, I'm building khush chatbot.
Summary: ASSISTANT:Great! What extra features do i need? USER:I want truncation and periodic summarization. ASSISTANT:Do i want summaries to replace my [...]
Summary: USER:Yes, and keep an archive. ASSISTANT:lets build one. USER:Yes, i am excited to build one ASSISTANT:yes me too lets gooooooo!

--- Archive ---
Archived #1:
   assistant: Great! What extra features do i need?
   user: I want truncation and periodic summarization.
   assistant: Do i want summaries to replace my old messages?
Archived #2:
   user: Yes, and keep an archive.
   assistant: lets build one.
   user: Yes, i am excited 